In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-08-31 12:12:06.446844: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-31 12:12:06.470660: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-08-31 12:12:07.458687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-31 12:12:07.458907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-31 12:12:07.458956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-08-31 12:12:07.710936: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-31 12:12:07.711056: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-31 12:12:07.711111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train', 'transactions.parquet'),engine="parquet",  part_mem_fraction=0.1)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid', 'transactions.parquet'),engine="parquet", part_mem_fraction=0.1)

In [3]:
customer_features = ['customer_id', 'age', 'FN', 'Active', 'club_member_status','fashion_news_frequency']
article_features = ['article_id', 'product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']

In [4]:
articles = cudf.read_parquet(os.path.join(config.data_raw_dir,'articles.parquet'))[article_features]
customers = cudf.read_parquet(os.path.join(config.data_raw_dir,'customers_enc.parquet'))[customer_features]

In [5]:
articles = nvt.Dataset(articles)
customers = nvt.Dataset(customers)

In [6]:
train = Dataset.merge(train, articles, on="article_id", how="inner")
train = Dataset.merge(train, customers, on="customer_id", how="inner")
valid = Dataset.merge(valid, articles, on="article_id", how="inner")
valid = Dataset.merge(valid, customers, on="customer_id", how="inner")
del articles, customers

In [7]:
# ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id','week']

article_id = ["article_id"] >> Categorify() >> TagAsItemID()
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()


price = ["price"] >> FillMissing(fill_val=0) >> Normalize() >> TagAsItemFeatures() >> Rename(postfix='_norm')

time_features = ["t_dat"]
session_time = (
    time_features >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

day = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.day)
        >> nvt.ops.Rename(name = 'day')
        >> Categorify()
        >> TagAsUserFeatures())
week = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.weekday)
        >> nvt.ops.Rename(name = 'week')
        >> Categorify()
        >> TagAsUserFeatures())
month = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.month)
        >> nvt.ops.Rename(name = 'month')
        >> Categorify()
        >> TagAsUserFeatures())
year = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.year)
        >> nvt.ops.Rename(name = 'year')
        >> Categorify()
        >> TagAsUserFeatures())


In [8]:
age_boundaries = list(np.arange(0,100,1))
age =  ['age'] >>  FillMissing(0) >> Bucketize(age_boundaries) >> Categorify() >> TagAsUserFeatures() 
# cat_customer_columns = (['FN', 'Active', 'club_member_status','fashion_news_frequency'] 
#             >> Categorify() 
#             >> TagAsUserFeatures()
#             )


In [9]:
# article_features = ['article_id', 'product_code', 'prod_name', 'product_type_no',
#        'product_type_name', 'product_group_name', 'graphical_appearance_no',
#        'graphical_appearance_name', 'colour_group_code', 'colour_group_name',
#        'perceived_colour_value_id', 'perceived_colour_value_name',
#        'perceived_colour_master_id', 'perceived_colour_master_name',
#        'department_no', 'department_name', 'index_code', 'index_name',
#        'index_group_no', 'index_group_name', 'section_no', 'section_name',
#        'garment_group_no', 'garment_group_name', 'detail_desc']

# cat_article_columns = (article_features 
#               >> Categorify()  
#               >> TagAsItemFeatures()
#               >> Rename(postfix='_cat'))

In [10]:
# features = article_id + customer_id + price + day + week + month + year + cat_customer_columns + age + cat_article_columns + time_features
# wf = nvt.Workflow(features)
# train = wf.fit_transform(train)
# valid = wf.transform(valid)
# train.to_parquet(os.path.join(config.data_processed_dir,'train_processed.parquet'))
# valid.to_parquet(os.path.join(config.data_processed_dir,'validation_processed.parquet'))

In [11]:
SESSIONS_MAX_LENGTH =2

# groupby_features = features >> Groupby(
#     groupby_cols=["customer_id"], 
#     sort_cols=["t_dat"],
#     aggs={'article_id': ['list', 'count'],
#         'price_norm': ['list', 'mean'],
#         'day': ['list'],
#         'week': ['list'],
#         'month': ['list'],
#         'year': ['list'],
#         'FN': ['list'],
#         'Active': ['list'],
#         'club_member_status': ['list'],
#         'fashion_news_frequency': ['list'],
#         'article_id_cat': ['list'],
#         'product_code_cat': ['list'],
#         'product_type_no_cat': ['list'],
#         'graphical_appearance_no_cat': ['list'],
#         'colour_group_code_cat': ['list'],
#         'perceived_colour_value_id_cat': ['list'],
#         'perceived_colour_master_id_cat': ['list'],
#         'department_no_cat': ['list'],
#         'index_code_cat': ['list'],
#         'index_group_no_cat': ['list'],
#         'section_no_cat': ['list'],
#         'garment_group_no_cat': ['list']},
#     name_sep="-")
features = article_id + customer_id + price + day + week + month + year + time_features
groupby_features = features >> Groupby(
    groupby_cols=["customer_id"], 
    sort_cols=["t_dat"],
    aggs={'article_id': ['list', 'count'],

        },
    name_sep="_")
groupby_features = groupby_features[] >> ListSlice(-SESSIONS_MAX_LENGTH) 
groupby_features = groupby_features + age + customer_id 


In [12]:
wf = nvt.Workflow(groupby_features)
train = wf.fit_transform(train)

Failed to transform operator <nvtabular.ops.list_slice.ListSlice object at 0x7f6b48376f10>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/merlin/dag/executors.py", line 237, in _run_node_transform
    transformed_data = node.op.transform(selection, input_data)
  File "/usr/local/lib/python3.8/dist-packages/nvtx/nvtx.py", line 101, in inner
    result = func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/nvtabular/ops/list_slice.py", line 106, in transform
    offsets = c.offsets.values
AttributeError: 'NumericalColumn' object has no attribute 'offsets'


AttributeError: 'NumericalColumn' object has no attribute 'offsets'

In [ ]:
train.head()

,article_id-list,age,customer_id
0,"[151, 2055]",4,97302
1,"[193, 25]",4,97302
2,"[9712, 12785]",4,97302
3,"[4425, 6450]",4,97302
4,"[4046, 5260]",4,97302


In [ ]:
wf.output_schema


,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,article_id-list,"(Tags.LIST, Tags.ITEM, Tags.CATEGORICAL, Tags.ID)","DType(name='int64', element_type=<ElementType....",True,True,None,0,0,.//categories/unique.article_id.parquet,0,55240,article_id,55241,512,0.0,2.0
1,age,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.age.parquet,0,87,age,88,20,NaN,NaN
2,customer_id,"(Tags.ID, Tags.CATEGORICAL, Tags.USER)","DType(name='int64', element_type=<ElementType....",False,False,None,0,0,.//categories/unique.customer_id.parquet,0,780123,customer_id,780124,512,NaN,NaN
